# Contract analytics for Cheese Grotto

In [2]:
import numpy as np
import pandas as pd
import requests
import time
import matplotlib.pyplot as plt
import seaborn as sns
from pandasql import sqldf #for manipulating DataFrames with SQL style queries
#Write file to machine to work with offline
import json

#ALL BELOW to export results of API request to local file
# Make it work for Python 2+3 and with Unicode
import io
try:
    to_unicode = unicode
except NameError:
    to_unicode = str
    
from tqdm.autonotebook import tqdm

from datetime import datetime

In [3]:
#generated each session by the graph api explorer
user_access_token = ''
insta_id = ''

# Instagram graph api

In [ ]:
#Get the user's pages
#For better security these must all be derived after a new user access token is created by the person logged 
#into the fb business account
user_pages = f'https://graph.facebook.com/v4.0/me/accounts?access_token={user_access_token}'
response = requests.get(user_pages)
page_id = json.loads(response.text)['data'][0]['id']

#Get the page's instagram business account id
insta_biz_account = f'https://graph.facebook.com/v4.0/{page_id}?fields=instagram_business_account&access_token={user_access_token}'
response = requests.get(insta_biz_account)
insta_id = json.loads(response.text)['instagram_business_account']['id']

In [ ]:
#Get the Instagram Business Account's Media Objects
media_objects = f'https://graph.facebook.com/v4.0/{insta_id}/media?access_token={user_access_token}&limit=2000'
response = requests.get(media_objects)
media_object_ids = [object['id'] for object in json.loads(response.text)['data']]

In [ ]:
len(json.loads(response.text)['data'])

In [ ]:
# media_objects = []
# for obj_id in media_object_ids:
#     media_objects.append(requests.get(f'https://graph.facebook.com/v4.0/{obj_id}?fields=comments,like_count,timestamp&access_token={user_access_token}').text)
#     time.sleep(1.5)

In [ ]:
#Get basic data on all media objects
media_objects_all = []
for obj_id in media_object_ids:
    media_objects_all.append(requests.get(f'https://graph.facebook.com/v4.0/{obj_id}?fields=caption,comments,comments_count,media_type,media_url,like_count,timestamp&access_token={user_access_token}').text)
    time.sleep(1.25)
    if len(media_objects_all) % 100 == 0:
        print(len(media_objects_all))

In [ ]:
len(media_objects_all)

In [ ]:
#SEe for more fields: https://developers.facebook.com/docs/instagram-api/reference/media/

requests.get(f'https://graph.facebook.com/v4.0/{media_object_ids[0]}?fields=caption,comments,comments_count,media_type,media_url,like_count,timestamp&access_token={user_access_token}').text
#/?fields={fields}

In [ ]:
# Write JSON file
with io.open('media_objects.json', 'w', encoding='utf8') as outfile:
    str_ = json.dumps(media_objects_all,
                      indent=4, sort_keys=True,
                      separators=(',', ': '), ensure_ascii=False)
    outfile.write(to_unicode(str_))

# Start here to get local data for df

In [3]:
# Read JSON file
with open('media_objects.json') as data_file:
    media_objects_json = json.load(data_file)

In [ ]:
media_objects_json[-1]

In [ ]:
#Only one thing has no caption?
[obj  for obj in media_objects_dicts if not 'caption' in list(obj.keys())]

In [45]:
#Get columns of data from json to build dataframe
media_objects_dicts = [json.loads(obj) for obj in media_objects_json]
columns = (list(media_objects_dicts[0].keys()))
data = {col: [obj[col] if col in list(obj.keys()) else None for obj in media_objects_dicts] for col in columns}

#build pandas data frame to look at data
df = pd.DataFrame(data)
df.set_index(pd.to_datetime(df['timestamp']), inplace = True)
#df['date_ordinal'] = pd.to_datetime(df['timestamp']).apply(lambda date: date.toordinal())
df.drop(['timestamp'], axis = 1, inplace = True)
df.sort_index(axis = 0, ascending = True, inplace = True)

In [46]:
df.head()

,caption,comments,comments_count,media_type,media_url,like_count,id
timestamp,,,,,,,
2014-11-10 22:47:02+00:00,"@CheeseGrotto on a crisp fall eve, in a Manhat...",{'data': [{'timestamp': '2014-11-11T00:20:51+0...,1,IMAGE,https://scontent.xx.fbcdn.net/v/t51.2885-15/10...,8,17841922594041130
2014-11-10 22:52:51+00:00,"Our #cheesesafe is a beautiful, simple gift fo...",None,0,IMAGE,https://scontent.xx.fbcdn.net/v/t51.2885-15/10...,6,17841735106041130
2014-11-10 23:04:47+00:00,This rich bloomy rind cheese was made in a @Ch...,{'data': [{'timestamp': '2016-09-09T03:22:34+0...,2,IMAGE,https://scontent.xx.fbcdn.net/v/t51.2885-15/10...,10,17843576941041130
2014-11-10 23:19:00+00:00,Our #cheesesafe with an awesome leather handle...,None,0,IMAGE,https://scontent.xx.fbcdn.net/v/t51.2885-15/10...,8,17841735103041130
2014-11-10 23:43:33+00:00,The goats want to know.....where's the @Cheese...,None,0,IMAGE,https://scontent.xx.fbcdn.net/v/t51.2885-15/10...,1,17843576947041130


In [ ]:
fig = plt.figure(figsize = (14,8))
plt.scatter(df.index, df['like_count'])
plt.title('Number of likes on posts over time')

In [ ]:
# fig = plt.figure(figsize = (8,6))
# plt.scatter(df.index, df['comments_count'])
# plt.title('Number of comments on posts over time')

In [ ]:
boostio = df['6/1/2018': '5/16/2019']
not_boostio = df.drop(boostio.index, axis = 0)

In [ ]:
#Drop the outliers that float higher than the rest
not_boostio = not_boostio[not_boostio['like_count'] < 350]
#Plot the plot
fig = plt.figure(figsize = (14,8))
plt.scatter(not_boostio.index, not_boostio['like_count'])
plt.title('Number of likes on posts over time, without Boostio')

first use of boostio: august 5 2018

last use of boostio: april 13. probably mid may

In [ ]:
#Drop the outliers that float lower than the rest
boostio = boostio[boostio['like_count'] > 200]
fig = plt.figure(figsize = (12,6))
plt.scatter(boostio.index, boostio['like_count'])
plt.title('Number of likes on posts over time with Boostio')

In [ ]:
boostio.describe()

In [ ]:
not_boostio.describe()

In [ ]:
#Show trend over time
not_b_index = np.arange(not_boostio['like_count'].values.shape[0])
fig = plt.figure(figsize = (16,8))
sns.regplot(x = not_b_index, 
            y = not_boostio['like_count'].values,
           scatter_kws=None, line_kws={'color':'red'})
plt.title('Likes over time, excluding Boostio')

# Caption feature engineering

In [ ]:
# boostio = df['6/1/2018': '5/16/2019']
# not_boostio = df.drop(boostio.index, axis = 0)

In [ ]:
df.head()

In [ ]:
df['10-11-2014':'10-11-2014']['caption']

In [ ]:
df['11-10-2014':'11-10-2014']['caption']

In [ ]:
df['11-10-2014':'11-10-2014'].index

In [ ]:
df[:'2014-11-10 23:53:32']['caption'].values

# Insights on media objects

In [ ]:
df['media_type'].value_counts()

# Story insights

In [ ]:
#Attempt to get stories
stories = f'https://graph.facebook.com/v4.0/{insta_id}/stories?access_token={user_access_token}&limit=2000'
response = requests.get(stories)
story_ids = [obj['id'] for obj in json.loads(response.text)['data']]

In [ ]:
#GEt timestamps on stories
story_timestamps = []
for obj_id in tqdm(story_ids):
    story_timestamps.append(requests.get(f'https://graph.facebook.com/v4.0/{obj_id}?fields=timestamp&access_token={user_access_token}').text)
    time.sleep(1.25)
    if len(story_timestamps) % 100 == 0:
        print(len(story_timestamps))

In [ ]:
#Get insights on stories
story_metrics = 'exits,impressions,reach,replies,taps_forward,taps_back'
story_insights = []
for obj_id in tqdm(story_ids):
    story_insights.append(requests.get(f'https://graph.facebook.com/v4.0/{obj_id}/insights?metric={story_metrics}&access_token={user_access_token}').text)
    time.sleep(1.25)
    if len(story_insights) % 100 == 0:
        print(len(story_insights))

In [ ]:
story_insights_json = [json.loads(obj) for obj in story_insights]
story_insight_fields = [story['name'] for story in story_insights_json[0]['data']]
#Bring relevant data together into a dict of lists 
story_insight_data = {field: [story['data'][i]['values'][0]['value'] for story in story_insights_json[:-1]]   for i, field in enumerate(story_insight_fields)}
#Add timestamp column
story_insight_data['timestamp'] = [json.loads(stamp)['timestamp'] for stamp in story_timestamps[:-1]]
#Description of story insight fields
story_insight_field_descriptions = [obj['name'] + ': ' + obj['description'] for obj in story_insights_json[0]['data']]
story_insight_field_descriptions

In [ ]:
#build pandas data frame to look at data
story_df = pd.DataFrame(story_insight_data)
story_df.set_index(pd.to_datetime(story_df['timestamp']), inplace = True)
story_df.drop(['timestamp'], axis = 1, inplace = True)
story_df.sort_index(axis = 0, ascending = True, inplace = True)

In [ ]:
story_df

In [ ]:
story_df.to_csv('story_insights.csv')

# Image insights

In [ ]:
#Get image insights
# image_metrics = 'engagement,impressions,reach,saved'
# image_ids = df[df['media_type'] == 'IMAGE']['id'].values
# image_insights = []
# for obj_id in tqdm(image_ids):
#     image_insights.append(requests.get(f'https://graph.facebook.com/v4.0/{obj_id}/insights?metric={image_metrics}&access_token={user_access_token}').text)
#     time.sleep(1.25)

In [ ]:
# Write JSON file
# with io.open('image_insights.json', 'w', encoding='utf8') as outfile:
#     str_ = json.dumps(image_insights,
#                       indent=4, sort_keys=True,
#                       separators=(',', ': '), ensure_ascii=False)
#     outfile.write(to_unicode(str_))

### Resume here to read in local file

In [ ]:
# Read JSON file
with open('image_insights.json') as data_file:
    image_insights_json = json.load(data_file)
    #Entries from before index 658 returned errors because the account wasn't a business account yet
    image_insights = [json.loads(obj) for obj in image_insights_json[658:]]

In [ ]:
image_insight_field_descriptions = [obj['name'] + ': ' + obj['description'] for obj in image_insights[0]['data']]
image_insight_field_descriptions

In [ ]:
image_insight_fields = [obj['name'] for obj in image_insights[0]['data']]
#Bring relevant data together into a dict of lists 
image_insight_data = {field: [obj['data'][i]['values'][0]['value'] for obj in image_insights]\
                          for i, field in enumerate(image_insight_fields)}
#Get ids for images with insights
image_insight_ids_raw = [obj['data'][0]['id'] for obj in image_insights]
image_insight_ids = [_id.split('/')[0] for _id in image_insight_ids_raw]
image_insight_data['id'] = image_insight_ids
#Get timestamps from main df to match image insights by id
image_insight_data['timestamp'] = [df[df['id'] == _id].index.values[0] for _id in image_insight_ids]
image_insight_data['like_count'] = [df[df['id'] == _id]['like_count'].values[0] for _id in image_insight_ids]
#Build df
image_insight_df = pd.DataFrame(image_insight_data)
#Rest index to timestamp
image_insight_df.set_index(pd.to_datetime(image_insight_df['timestamp']), inplace = True)
image_insight_df.drop(['timestamp'], axis = 1, inplace = True)

In [ ]:
image_insight_df.head()

In [ ]:
image_insight_df[['like_count', 'engagement']]

In [ ]:
#Write image insights df to csv for testing with spyre
image_insight_df.to_csv('image_insights.csv')

In [ ]:
fig = plt.figure(figsize = (14,8))
plt.plot(image_insight_df.index, image_insight_df['like_count'], label = 'likes count')
plt.plot(image_insight_df.index, image_insight_df['engagement'], label = 'engagement')
plt.plot(image_insight_df.index, image_insight_df['saved'], label = 'saved')
plt.axvspan('6/1/2018','5/16/2019', color ='lightgrey', label = 'Start and end of boostio')
plt.title('Image insights: engagement and saved')
plt.legend()
plt.show()

In [ ]:
fig = plt.figure(figsize = (14,8))
plt.plot(image_insight_df.index, image_insight_df['like_count'], label = 'likes count')
plt.plot(image_insight_df.index, image_insight_df['impressions'], label = 'impressions')
plt.plot(image_insight_df.index, image_insight_df['reach'], label = 'reach')
plt.axvspan('6/1/2018','5/16/2019', color ='lightgrey', label = 'Start and end of boostio')
plt.legend()
plt.title('Image insights: impressions and reach')
plt.show()

# Video insights

In [ ]:
video_metrics = photo_metrics + 'video_views'
video_ids = df[df['media_type'] == 'VIDEO']['id'].values

In [ ]:
#Get video ids
stories = f'https://graph.facebook.com/v4.0/{insta_id}/stories?access_token={user_access_token}&limit=2000'
response = requests.get(stories)
story_ids = [obj['id'] for obj in json.loads(response.text)['data']]

# Album insights

In [ ]:
album_metrics = 'carousel_album_engagement,carousel_album_impressions,carousel_album_reach,carousel_album_saved,carousel_album_video_views'
album_ids = df[df['media_type'] == 'CAROUSEL_ALBUM']['id'].values

# "Lifetime" user insights

In [46]:
lifetime_metrics = 'audience_city,audience_country,audience_gender_age,audience_locale,online_followers,'

In [47]:
response = requests.get(f'https://graph.facebook.com/v4.0/{insta_id}/insights?metric={lifetime_metrics}&period=lifetime&{user_access_token}')

In [ ]:
response.text

In [48]:
lifetime_data = json.loads(response.text)

In [49]:
lifetime_data['data']

[{'name': 'audience_city',
  'period': 'lifetime',
  'values': [{'value': {'Pittsburgh, Pennsylvania': 37,
     'Sydney, New South Wales': 76,
     'London, England': 107,
     'Philadelphia, Pennsylvania': 123,
     'São Paulo, São Paulo (state)': 37,
     'Atlanta, Georgia': 58,
     'Jacksonville, Florida': 30,
     'Oakland, California': 32,
     'Seattle, Washington': 67,
     'Cambridge, Massachusetts': 34,
     'Phoenix, Arizona': 50,
     'Austin, Texas': 61,
     'Izmir, İzmir Province': 33,
     'Ahmedabad, Gujarat': 42,
     'Moscow, Moscow': 36,
     'Chicago, Illinois': 128,
     'Jakarta, Jakarta': 73,
     'Paris, Île-de-France': 38,
     'Buenos Aires, Ciudad Autónoma de Buenos Aires': 40,
     'Salt Lake City, Utah': 34,
     'Baku, Baku': 75,
     'Melbourne, Victoria': 102,
     'Delhi, Delhi': 45,
     'Los Angeles, California': 236,
     'Las Vegas, Nevada': 56,
     'Brisbane, Queensland': 34,
     'Dallas, Texas': 46,
     'Washington, Washington, District of Col

In [71]:
first_day = 1415577600
interval = 2592000
today = 1572652800
yesterday = 1572566400
halloween = 1572480000
since = yesterday - interval
until = yesterday

In [72]:
request = 'https://graph.facebook.com/v4.0/' + insta_id
request += '/insights?metric=' + lifetime_metrics
request += '&period=lifetime'
request += '&since=' + str(halloween - interval)
request += '&until=' + str(halloween)
request += '&' + user_access_token

In [73]:
response = requests.get(request)
lifetime_data = json.loads(response.text)

In [74]:
lifetime_data

{'error': {'message': '(#100) (audience_city) metric supports querying data only till yesterday',
  'type': 'OAuthException',
  'code': 100,
  'fbtrace_id': 'A1sSYRZRSgKqQNy4VKG1EBE'}}

In [ ]:
# Write JSON file
with io.open('dailies.json', 'w', encoding='utf8') as outfile:
    str_ = json.dumps(dailies_complete,
                      indent=4, sort_keys=True,
                      separators=(',', ': '), ensure_ascii=False)
    outfile.write(to_unicode(str_))

# "Daily" insights

In [ ]:
day_metrics = 'email_contacts,follower_count,get_directions_clicks,impressions,phone_call_clicks,profile_views,reach,text_message_clicks,website_clicks'

first_day = 1415577600
interval = 2592000
today = 1572652800

dailies = []

since = today - interval
until = today
while since >= first_day:
    request = 'https://graph.facebook.com/' + insta_id
    request += '/insights?metric=' + day_metrics
    request += '&period=day&since=' + str(since)
    request += '&until=' + str(until)
    request += '&' + user_access_token
    
    response = requests.get(request)
    
    dailies.append(json.loads(response.text))
    
    until -= interval
    since -= interval
    
    time.sleep(1.5)

In [3]:
# Read JSON file
with open('dailies.json') as data_file:
    dailies_json = json.load(data_file)

In [ ]:
#Create timestamp index
dailies_json.reverse()
timestamp = []
for daily in dailies_json:
    timestamp.extend([value['end_time'] for value in daily['data'][0]['values']])

In [30]:
#Get all data to build df
metrics = []
for metric in dailies_json[0]['data']:
    metrics.append(metric['name'])
data = {metric: [] for metric in metrics}

[data[metric['name']].extend([value['value'] for value in metric['values']]) for daily in dailies_json for metric in daily['data']]

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,

In [37]:
#build pandas data frame to look at data
dailies_df = pd.DataFrame(data)
dailies_df.set_index(pd.to_datetime(timestamp), inplace = True)

In [39]:
#Write all dailies to csv to be transferred to google sheet
dailies_df['2017-10-31 07:00:00+00:00':].to_csv(path_or_buf = 'dailies.csv')